In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
valid_ratio=0.2
batch_size = 16
epochs = 15

In [56]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np

In [34]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0.5, 0.5)])

In [37]:
train_valid_dataset = torchvision.datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)


test_dataset = torchvision.datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


In [52]:
nb_train = int((1.0 - valid_ratio) * len(train_valid_dataset))
nb_valid = int(valid_ratio * len(train_valid_dataset))
train_dataset, dev_dataset = torch.utils.data.dataset.random_split(
    train_valid_dataset, [nb_train, nb_valid]
)

train_X = train_dataset.dataset.data[train_dataset.indices]
dev_X = dev_dataset.dataset.data[dev_dataset.indices]
test_X = test_dataset.data

In [39]:
train_y = train_dataset.dataset.targets[train_dataset.indices].numpy() - 1
dev_y = dev_dataset.dataset.targets[dev_dataset.indices].numpy() - 1
test_y = test_dataset.targets.numpy() - 1

In [40]:
n_classes = len(set(train_dataset.dataset.targets.numpy()))

In [41]:

train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [82]:
from conformalSparsemax import Classifier
classifier = Classifier()

In [83]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)

In [84]:

for epoch in range(epochs):  # loop over the dataset multiple times

    train_losses = []
        # zero the parameter gradients
    for i, data in tqdm(enumerate(train_dataloader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_losses.append(loss)
    mean_loss = torch.tensor(train_losses).mean().item()
    print(f'Epoch {epoch + 1} loss: {mean_loss:.3f}')
    predicted_labels=[]
    true_labels = []
    with torch.no_grad():
        for data in dev_dataloader:
            x, y = data
            predicted_labels.append(classifier(x).argmax(dim=-1).numpy())
            true_labels.append(y.numpy())
                
    predicted_labels = np.concatenate(predicted_labels)
    true_labels = np.concatenate(true_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    print(f'Epoch {epoch + 1} f1: {f1:.3f}')

print('Finished Training')

3000it [00:52, 57.22it/s]


Epoch 1 loss: 0.419
Epoch 1 f1: 0.959


3000it [00:52, 56.83it/s]


Epoch 2 loss: 0.097
Epoch 2 f1: 0.977


3000it [00:51, 58.40it/s]


Epoch 3 loss: 0.062
Epoch 3 f1: 0.980


3000it [00:49, 60.27it/s]


Epoch 4 loss: 0.045
Epoch 4 f1: 0.983


228it [00:03, 58.57it/s]